In [21]:
#Firstly importing all  the necessary libraries which will be used
import tensorflow as tf
print("The tensorflow version which we will be using is {}".format(tf.__version__))
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout,Conv2D
import cv2
import os
import pickle
import numpy as np
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline


The tensorflow version which we will be using is 1.12.0


In [22]:
training_data=[]
def create_training_data(): 
    #Now defining the base directory from where we will load our data, this will be our base directory from where we will load our data
    BASEDIR="E:\deep-learning\datasets\kagglecatsanddogs_3367a\PetImages"
    #the two categories of the data which we have
    categories=["Dog","Cat"]
    #The size of the image which we will be using
    IMG_SIZE=100
    #the index which we will be using
    class_num=[]
    for category in categories:
        full_path=os.path.join(BASEDIR,category)
        class_num=categories.index(category)
        for file in os.listdir(full_path):
            full_file_name=os.path.join(full_path,file)
            try:
                img_array=cv2.imread(full_file_name,cv2.IMREAD_GRAYSCALE)
                new_img_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                training_data.append([new_img_array,class_num])
            except Exception as e:
                pass


In [23]:
create_training_data()
print("The length of the training data is {}".format(len(training_data)))

The length of the training data is 24946


In [24]:
#Now shuffling and extracting features and the labels
IMG_SIZE=100
def extract_features_labels():
    import random
    random.shuffle(training_data)
    #Now extracting features and labels
    X=[]
    Y=[]
    for features,labels in training_data:
        X.append(features)
        Y.append(labels)
    X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1) #1 at the last as it's ann greyscale image
    Y=np.array(Y)
    print("The shape of the Features are {}".format(X.shape))
    print("The shape of the labels are {}".format(Y.shape))
    #Normalizing our data
    X=X/255
    return X,Y
X,Y=extract_features_labels()

The shape of the Features are (24946, 100, 100, 1)
The shape of the labels are (24946,)


In [25]:
"""#Saving our data
import pickle
pickle_out=open("X.pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out=open("Y.pickle","wb")
pickle.dump(Y,pickle_out)
pickle_out.close()"""

'#Saving our data\nimport pickle\npickle_out=open("X.pickle","wb")\npickle.dump(X,pickle_out)\npickle_out.close()\n\npickle_out=open("Y.pickle","wb")\npickle.dump(Y,pickle_out)\npickle_out.close()'

In [26]:
"""pickle_in=open("X.pickle","rb")
X=pickle.load(pickle_in)
pickle_in=open("Y.pickle","rb")
Y=pickle.load(pickle_in)"""

'pickle_in=open("X.pickle","rb")\nX=pickle.load(pickle_in)\npickle_in=open("Y.pickle","rb")\nY=pickle.load(pickle_in)'

In [33]:
from tensorflow.keras.layers import Activation,MaxPooling2D
def create_model():

    dense_layers = [0]
    layer_sizes = [64]
    conv_layers = [3]

    for dense_layer in dense_layers:
        for layer_size in layer_sizes:
            for conv_layer in conv_layers:
                NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
                print(NAME)

                model = Sequential()

                model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

                for l in range(conv_layer-1):
                    model.add(Conv2D(layer_size, (3, 3)))
                    model.add(Activation('relu'))
                    model.add(MaxPooling2D(pool_size=(2, 2)))

                model.add(Flatten())

                for _ in range(dense_layer):
                    model.add(Dense(layer_size))
                    model.add(Activation('relu'))

                model.add(Dense(1))
                model.add(Activation('sigmoid'))

                tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

                model.compile(loss='binary_crossentropy',
                              optimizer='adam',
                              metrics=['accuracy'],
                              )

                model.fit(X, Y,
                          batch_size=32,
                          epochs=10,
                          validation_split=0.3,
                          callbacks=[tensorboard])

    model.save('64x3-CNN.model')


In [34]:
create_model()

3-conv-64-nodes-0-dense-1546459786
Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 139s 8ms/step - loss: 0.6522 - acc: 0.6038 - val_loss: 0.5749 - val_acc: 0.7043
Epoch 2/10
17462/17462 [==============================] - 125s 7ms/step - loss: 0.5438 - acc: 0.7273 - val_loss: 0.5053 - val_acc: 0.7505
Epoch 3/10
17462/17462 [==============================] - 123s 7ms/step - loss: 0.4746 - acc: 0.7748 - val_loss: 0.5110 - val_acc: 0.7408
Epoch 4/10
17462/17462 [==============================] - 123s 7ms/step - loss: 0.4287 - acc: 0.8012 - val_loss: 0.4366 - val_acc: 0.8012
Epoch 5/10
17462/17462 [==============================] - 126s 7ms/step - loss: 0.3922 - acc: 0.8233 - val_loss: 0.4372 - val_acc: 0.7998
Epoch 6/10
17462/17462 [==============================] - 133s 8ms/step - loss: 0.3588 - acc: 0.8389 - val_loss: 0.4190 - val_acc: 0.8156
Epoch 7/10
17462/17462 [==============================] - 125s 7ms/step - loss: 0.3273 -

In [2]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]  # will use this to convert prediction num to string value


def prepare(filepath):
    IMG_SIZE = 100  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # return the image with shaping that TF wants.

E:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
model = tf.keras.models.load_model("64x3-CNN.model")
prediction = model.predict([prepare('dog.jpg')])  # REMEMBER YOU'RE PASSING A LIST OF THINGS YOU WISH TO PREDICT
image = cv2.imread('dog.jpg',0)
cv2.imshow('image',image)
print(CATEGORIES[int(prediction[0][0])])
cv2.waitKey(0)
cv2.destroyAllWindows()